In [ ]:
import random
import pandas as pd
from datetime import datetime, timedelta

# Define constraints
num_cars = 30
num_rentals = 300
num_renters = 100  # Restricting renter_id between 1-100

# Generate rental data
rental_data = []
car_rental_dates = {car_id: [] for car_id in range(1, num_cars + 1)}  # Track car rental periods

for rental_id in range(1, num_rentals + 1):
    car_id = random.randint(1, num_cars)  # Assign a random car

    # Generate a valid rental period ensuring no overlapping rentals per car
    while True:
        start_date = datetime(2024, random.randint(1, 12), random.randint(1, 28))  # Random month/day
        end_date = start_date + timedelta(days=random.randint(1, 14))  # Rental lasts 1-14 days

        # Ensure no overlapping rentals for the same car
        overlapping = any(
            (start <= end_date and end >= start_date) for start, end in car_rental_dates[car_id]
        )
        
        if not overlapping:
            car_rental_dates[car_id].append((start_date, end_date))
            break

    renter_id = random.randint(1, num_renters)  # Assign a valid renter (1-100)

    # Format dates in ISO 8601 (YYYYMMDD)
    start_date_str = start_date.strftime('%Y%m%d')
    end_date_str = end_date.strftime('%Y%m%d')

    rental_data.append((rental_id, car_id, renter_id, start_date_str, end_date_str))

# Create a DataFrame
df_rental = pd.DataFrame(rental_data, columns=["rental_id", "car_id", "renter_id", "start_date", "end_date"])

# Save to CSV (store it locally)
csv_filename = "rental_data_iso8601.csv"
df_rental.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been saved successfully!")


CSV file 'rental_data_iso8601.csv' has been saved successfully!


In [4]:
# Read the CSV file
df_rental = pd.read_csv("rental_data_iso8601.csv") 

# Convert date format from YYYYMMDD to MM/DD/YYYY
df_rental["start_date"] = pd.to_datetime(df_rental["start_date"], format='%Y%m%d').dt.strftime('%-m/%-d/%Y')
df_rental["end_date"] = pd.to_datetime(df_rental["end_date"], format='%Y%m%d').dt.strftime('%-m/%-d/%Y')

# Generate SQL INSERT statements
sql_statements = [
    f"INSERT INTO rental (rental_id, car_id, renter_id, start_date, end_date) "
    f"VALUES ({row.rental_id}, {row.car_id}, {row.renter_id}, '{row.start_date}', '{row.end_date}');"
    for _, row in df_rental.iterrows()
]

# Save to a .sql file
sql_filename = "rental_data.sql"
with open(sql_filename, "w") as f:
    f.write("\n".join(sql_statements))

print(f"SQL file '{sql_filename}' has been generated successfully!")


SQL file 'rental_data.sql' has been generated successfully!
